# Lesson 3: Connecting to a SQL Database

## Setup

In [2]:
import os
from IPython.display import Markdown, HTML, display
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain_openai import AzureChatOpenAI
import sqlite3

## Recover the original dataset

**Note**: To access the data locally, use the following code:

```
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
```

In [3]:
from sqlalchemy import create_engine
import pandas as pd

data_path = os.getenv("DATA_PATH")
csv_path = os.path.join(data_path, "all-states-history.csv")

df = pd.read_csv(csv_path, encoding='latin1', on_bad_lines='skip').fillna(value=0)
df

,date,state,death,hospitalized,negative,positive,recovered
0,3/7/2021,AK,305,1293.0,0.0,56886,0.0
1,3/7/2021,AL,10148,45976.0,1931711.0,499819,295690.0
2,3/7/2021,AR,5319,14926.0,2480716.0,324818,315517.0
3,3/7/2021,AS,0,0.0,2140.0,0,0.0
4,3/7/2021,AZ,16328,57907.0,3073010.0,826454,0.0
5,3/7/2021,CA,54124,0.0,0.0,3501394,0.0
6,3/7/2021,CO,5989,23904.0,2199458.0,436602,0.0
7,3/7/2021,CT,7704,0.0,0.0,285330,0.0
8,3/7/2021,DC,1030,0.0,0.0,41419,29570.0
9,3/7/2021,DE,1473,0.0,545070.0,88354,0.0


## Move the data to the SQL database

In [4]:
# Path to your SQLite database file
database_file_path = os.path.join(data_path, "test.db")

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')

df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False
)

21

## Prepare the SQL prompt

In [5]:
MSSQL_AGENT_PREFIX = """

You are an agent designed to interact with a SQL database.
## Instructions:
- Given an input question, create a syntactically correct {dialect} query
to run, then look at the results of the query and return the answer.
- Unless the user specifies a specific number of examples they wish to
obtain, **ALWAYS** limit your query to at most {top_k} results.
- You can order the results by a relevant column to return the most
interesting examples in the database.
- Never query for all the columns from a specific table, only ask for
the relevant columns given the question.
- You have access to tools for interacting with the database.
- You MUST double check your query before executing it.If you get an error
while executing a query,rewrite the query and try again.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.)
to the database.
- DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE, ONLY USE THE RESULTS
OF THE CALCULATIONS YOU HAVE DONE.
- Your response should be in Markdown. However, **when running  a SQL Query
in "Action Input", do not include the markdown backticks**.
Those are only for formatting the response, not for executing the command.
- ALWAYS, as part of your final answer, explain how you got to the answer
on a section that starts with: "Explanation:". Include the SQL query as
part of the explanation section.
- If the question does not seem related to the database, just return
"I don\'t know" as the answer.
- Only use the below tools. Only use the information returned by the
below tools to construct your query and final answer.
- Do not make up table names, only use the tables returned by any of the
tools below.

## Tools:

"""

In [6]:
MSSQL_AGENT_FORMAT_INSTRUCTIONS = """

## Use the following format:

Question: the input question you must answer.
Thought: you should always think about what to do.
Action: the action to take, should be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action.
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer.
Final Answer: the final answer to the original input question.

Example of Final Answer:
<=== Beginning of example

Action: query_sql_db
Action Input: 
SELECT SUM(death) FROM all_states_history

Observation:
[(27437.0,)]
Thought:I now know the final answer
Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `all_states_history` table for the `death` column where the state
is 'TX' and the date starts with '2020'. The query returned a list of tuples
with the number of deaths for each day in 2020. To answer the question,
I took the sum of all the deaths in the list, which is 27437.
I used the following query

```sql
SELECT SUM(death) FROM all_states_history'"
```
===> End of Example

"""

## Call the Azure Chat model and create the SQL agent

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [7]:
llm = AzureChatOpenAI(
    openai_api_version="2024-05-01-preview",
    azure_deployment="gpt-35-turbo",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    temperature=0, 
    max_tokens=500
)

db = SQLDatabase.from_uri(f'sqlite:///{database_file_path}')
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [8]:
QUESTION = """How many patients (total sum) were dead vs recovered from the whole dataset?
"""

agent_executor_SQL = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [9]:
#Check the content of the database

# Verify the table creation
conn = sqlite3.connect(database_file_path)
cursor = conn.cursor()

# List all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

# Query the database to check the content of the table
query = "SELECT * FROM all_states_history"
df_check = pd.read_sql_query(query, conn)

# Display the DataFrame
print(df_check)

# Close the connection
conn.close()

Tables in the database: [('all_states_history',)]
        date state  death  hospitalized   negative  positive  recovered
0   3/7/2021    AK    305        1293.0        0.0     56886        0.0
1   3/7/2021    AL  10148       45976.0  1931711.0    499819   295690.0
2   3/7/2021    AR   5319       14926.0  2480716.0    324818   315517.0
3   3/7/2021    AS      0           0.0     2140.0         0        0.0
4   3/7/2021    AZ  16328       57907.0  3073010.0    826454        0.0
5   3/7/2021    CA  54124           0.0        0.0   3501394        0.0
6   3/7/2021    CO   5989       23904.0  2199458.0    436602        0.0
7   3/7/2021    CT   7704           0.0        0.0    285330        0.0
8   3/7/2021    DC   1030           0.0        0.0     41419    29570.0
9   3/7/2021    DE   1473           0.0   545070.0     88354        0.0
10  3/7/2021    FL  32266       82237.0  9339038.0   1909209        0.0
11  3/7/2021    GA  17906       56797.0        0.0   1023487        0.0
12  3/7/2021  

## Invoke the SQL model

In [29]:
agent_executor_SQL.invoke(QUESTION, handle_parsing_errors=True)



> Entering new AgentExecutor chain...
I need to find the total sum of patients who were dead and the total sum of patients who recovered from the entire dataset.
Action: sql_db_query_checker
Action Input: 
SELECT SUM(death) AS total_dead, SUM(recovered) AS total_recovered FROM datasetSELECT SUM(death) AS total_dead, SUM(recovered) AS total_recovered FROM datasetThe query is correct.
Action: sql_db_query
Action Input: 
SELECT SUM(death) AS total_dead, SUM(recovered) AS total_recovered FROM datasetError: (sqlite3.OperationalError) no such table: dataset
[SQL: SELECT SUM(death) AS total_dead, SUM(recovered) AS total_recovered FROM dataset]
(Background on this error at: https://sqlalche.me/e/20/e3q8)The table 'dataset' does not exist. I need to check the available tables in the database.
Action: sql_db_list_tables
Action Input: 
""all_states_historyThe only table available in the database is 'all_states_history'.
Action: sql_db_query_checker
Action Input: 
SELECT SUM(death) AS total_dead

{'input': 'How many patients (total sum) were dead vs recovered from the whole dataset?\n',
 'output': 'There were 215,737 patients who were dead and 1,528,537 patients who recovered from the whole dataset.'}